In [1]:
import pandas as pd
import numpy as np
import sklearn as sk
import urllib
import math
%pylab inline

import findspark
findspark.init()

from pyspark import SparkContext
#sc.stop()
sc = SparkContext(master="local[3]",pyFiles=['lib/numpy_pack.py','lib/spark_PCA.py','lib/computeStats.py'])

from pyspark import SparkContext
from pyspark.sql import *
sqlContext = SQLContext(sc)

import sys
sys.path.append('./lib')

import numpy as np
from numpy_pack import packArray,unpackArray
from spark_PCA import computeCov
from computeStats import computeOverAllDist, STAT_Descriptions

### Read the data frame from pickle file

data_dir='../../Data/Weather'
file_index='SSSSBBBB'

from pickle import load

#read statistics
filename=data_dir+'/STAT_%s.pickle'%file_index
STAT,STAT_Descriptions = load(open(filename,'rb'))
print 'keys from STAT=',STAT.keys()

#read data
filename=data_dir+'/US_Weather_%s.parquet'%file_index

df=sqlContext.read.parquet(filename)
print df.count()
df.show(5)

Populating the interactive namespace from numpy and matplotlib
keys from STAT= ['TMIN', 'TOBS', 'TMAX', 'SNOW', 'SNWD', 'PRCP']
12740
+---------+--------+---------+-----------+-----------+------+--------------------+------+--------+
|elevation|latitude|longitude|measurement|    station|undefs|              vector|  year|   label|
+---------+--------+---------+-----------+-----------+------+--------------------+------+--------+
|   2449.1| 36.0333|-105.8167|       TMAX|USC00299113|     6|[40 56 30 54 40 5...|1910.0|SSSSBBBB|
|   2449.1| 36.0333|-105.8167|       TMAX|USC00299113|    21|[80 C9 30 D4 00 4...|1911.0|SSSSBBBB|
|   2449.1| 36.0333|-105.8167|       TMAX|USC00299113|     2|[E0 D0 80 C9 E0 D...|1912.0|SSSSBBBB|
|   2449.1| 36.0333|-105.8167|       TMAX|USC00299113|     5|[80 C9 00 C6 30 5...|1913.0|SSSSBBBB|
|   2449.1| 36.0333|-105.8167|       TMAX|USC00299113|     0|[80 CD 40 CC 00 4...|1914.0|SSSSBBBB|
+---------+--------+---------+-----------+-----------+------+-------------

In [3]:
sqlContext.registerDataFrameAsTable(df,'weather')
Query="SELECT station, latitude,longitude,elevation FROM weather"
print(Query)
df1 = sqlContext.sql(Query)
df1.show(4)

SELECT station, latitude,longitude,elevation FROM weather
+-----------+--------+---------+---------+
|    station|latitude|longitude|elevation|
+-----------+--------+---------+---------+
|USC00299113| 36.0333|-105.8167|   2449.1|
|USC00299113| 36.0333|-105.8167|   2449.1|
|USC00299113| 36.0333|-105.8167|   2449.1|
|USC00299113| 36.0333|-105.8167|   2449.1|
+-----------+--------+---------+---------+
only showing top 4 rows



In [4]:
from ipyleaflet import (
    Map,
    Marker,
    TileLayer, ImageOverlay,
    Polyline, Polygon, Rectangle, Circle, CircleMarker,
    GeoJSON,
    DrawControl
)

In [5]:
df2=df1.groupby(['station','latitude','longitude','elevation']).agg({"station": "count"})
pdf=df2.toPandas()
pdf.sort_values(by=['station'],inplace=True)
pdf.head(2)

,station,latitude,longitude,elevation,count(station)
114,US1NMBR0004,35.1296,-106.3204,2101.0,6
185,US1NMBR0006,35.1198,-106.6728,1517.0,8


In [6]:
min_lat,max_lat,min_long,max_long = box = (pdf['latitude'].min(), pdf['latitude'].max(), pdf['longitude'].min(), pdf['longitude'].max())

In [7]:
center = [(min_lat+max_lat)/2, (min_long+max_long)/2]
zoom = 9

m = Map(default_tiles=TileLayer(opacity=1.0), center=center, zoom=zoom)

r = Rectangle(bounds=[[min_lat,min_long],[max_lat,max_long]], weight=5, fill_opacity=0.0)
m += r

lat_margin=(max_lat-min_lat)/4
long_margin=(max_long-min_long)/4
circles = []
for index,row in pdf.iterrows():
    _lat=row['latitude']
    _long=row['longitude']
    _count=row['count(station)']
    # taking sqrt of count so that the  area of the circle corresponds to the count
    c = Circle(location=(_lat,_long), radius=int(300*np.sqrt(_count+0.0)), weight=1,
            color='#F00', opacity=0.8, fill_opacity=0.4)
    circles.append(c)
    m.add_layer(c)
m    